# Wrapping the model as a reddit bot

After training and saving the trained model, we want to wrap it as a reddit bot that produces a score for every post made on the subreddit r/ProperTechno as a post reply. We will use the Reddit API